In [3]:
import os
import pickle
import pytesseract
import torch
import cv2
import numpy as np
from flask import Flask, request, jsonify, render_template
from transformers import pipeline
from werkzeug.utils import secure_filename

app = Flask(__name__)
app.config['UPLOAD_FOLDER'] = 'uploads/'
app.config['MODEL_FOLDER'] = 'models/'

os.makedirs(app.config['UPLOAD_FOLDER'], exist_ok=True)
os.makedirs(app.config['MODEL_FOLDER'], exist_ok=True)

# Load LLaMA for summarization
summarizer = pipeline("summarization", model="meta-llama/Meta-Llama-3")

def extract_text_from_image(image_path):
    image = cv2.imread(image_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    text = pytesseract.image_to_string(gray)
    return text.strip()

def load_models():
    models = {}
    for filename in os.listdir(app.config['MODEL_FOLDER']):
        if filename.endswith('.pkl'):
            with open(os.path.join(app.config['MODEL_FOLDER'], filename), 'rb') as file:
                models[filename] = pickle.load(file)
    return models

@app.route('/')
def home():
    return render_template('index.html')

@app.route('/upload', methods=['POST'])
def upload_file():
    if 'file' not in request.files:
        return jsonify({"error": "No file part"})
    file = request.files['file']
    if file.filename == '':
        return jsonify({"error": "No selected file"})
    
    filename = secure_filename(file.filename)
    file_path = os.path.join(app.config['UPLOAD_FOLDER'], filename)
    file.save(file_path)
    
    extracted_text = extract_text_from_image(file_path)
    summary = summarizer(extracted_text, max_length=100, min_length=20, do_sample=False)[0]['summary_text']
    
    models = load_models()
    predictions = {}
    feature_importance = {}
    
    for model_name, model in models.items():
        try:
            pred = model.predict([extracted_text])
            predictions[model_name] = pred.tolist()
            if hasattr(model, 'feature_importances_'):
                feature_importance[model_name] = model.feature_importances_.tolist()
        except Exception as e:
            predictions[model_name] = f"Error in prediction: {str(e)}"
    
    heart_disease_risk = predictions.get("heart_disease_model.pkl", "No prediction available")
    
    return jsonify({
        "summary": summary,
        "predictions": predictions,
        "feature_importance": feature_importance,
        "heart_disease_risk": heart_disease_risk
    })

if __name__ == '__main__':
    app.run(debug=True)


ModuleNotFoundError: No module named 'transformers.utils'